In [6]:
#!python ../setup.py build_ext --inplace --force # sets up package under /redundancy
#!pip install hyppo # alternative impnt.

In [7]:
from time import sleep
%matplotlib inline

import multiprocessing as mp
import gc
from redundancy import hsic
import time
from matplotlib import pyplot as plt
from parallelqueue.base_models import *
from parallelqueue.monitors import *
ncpus = mp.cpu_count() - 12
print(ncpus)


class Concurrent:
    def __init__(self, maxtime=1000, rho = 0.9, d=2, r=2,
                 order=range(2, 1000, 2), seed=123):
        self.rhoJSim = None
        self.r = r
        self.order = order
        self.d = d
        self.rho = rho
        self.maxtime = maxtime
        self._ts = None
        self.seed = seed


    def WriteEach(self, simrep=1, of=["TSim"], ts=True):
        """                   v rep within run
        res['Thresh(2,2)'][0][0].keys()
                           ^run (N-size)
        Out[9]: dict_keys(['ReplicaSets', 'TimeQueueSize'])"""
        self._ts = ts
        self._sims = simrep
        labels = {self.RSim: f"Redundancy({self.d})", self.JSim: f"JSQ({self.d})",
                  self.TSim: f"Thresh({self.d},{self.r})"}
        self.res = {}
        for sim in [self.__getattribute__(i) for i in of]:
            print(f"Running {sim}")
            self.res[labels[sim]] = self.ParallelSim(sim)  # Annoying to unpack ¯\_(ツ)_/¯

    def DoEach(self, of=["TSim"], iters=1):
        self._ts = False
        labels = {self.RSim: f"Redundancy({self.d})", self.JSim: f"JSQ({self.d})",
                  self.TSim: f"Thresh({self.d},{self.r})"}
        self.res = {}
        for sim in [self.__getattribute__(i) for i in of]:
            print(f"Running {sim}")
            results = self.ParallelSim(sim)
            self.res[labels[sim]] = pd.DataFrame(results)
        for k, v in self.res.items():
            plt.plot(v, label=k)
        plt.legend()
        if iters == 1:
            plt.title("Average Time in System as N → ∞")

    def RSim(self, reps):
        mons = [TimeQueueSize]
        testvalues = []
        for N in (self.order):
            _sim = RedundancyQueueSystem(maxTime=self.maxtime, parallelism=N, seed=self.seed + 2331 * N + reps,
                                         d=self.d,
                                         Arrival=random.expovariate,
                                         AArgs=(self.rho*N)/self.d, Service=random.expovariate, SArgs=1,
                                         Monitors=mons)
            _sim.RunSim()
            testvalues.append(_sim.MonitorOutput)
        if not self._ts:
            return np.array(testvalues)
        else:
            return np.mean(testvalues)

    def JSim(self, reps):
        mons = [TimeQueueSize]
        testvalues = []
        for N in (self.order):
            _sim = JSQd(maxTime=self.maxtime, parallelism=N, seed=self.seed + 2331 * N + reps, d=self.d,
                        Arrival=random.expovariate,
                        AArgs=(self.rho*N)/self.d, Service=random.expovariate, SArgs=1,
                        Monitors=mons)
            _sim.RunSim()
            testvalues.append(_sim.MonitorOutput)
        if not self._ts:
            return np.array(testvalues)
        else:
            return np.mean(testvalues)

    def TSim(self, reps):
        mons = [TimeQueueSize]
        testvalues = []
        for N in (self.order):
            _sim = ParallelQueueSystem(maxTime=self.maxtime, parallelism=N, seed=self.seed + 2331 * N + reps,
                                         d=self.d,
                                         r=self.r,
                                         Arrival=random.expovariate,
                                         AArgs=(self.rho*N)/self.d, Service=random.expovariate, SArgs=1,
                                         Monitors=mons)
            _sim.RunSim()
            testvalues.append(_sim.MonitorOutput)
        if not self._ts:
            return np.array(testvalues)
        else:
            return np.mean(testvalues)


    def ParallelSim(self, sim):
        with mp.Pool(processes=ncpus) as p:
            res = p.map(sim, range(self._sims))
        return res

    def Results(self):
        return self.res


def SafeRun(maxtime=1000, rho = 0.9, d=2, r=2, order=range(2, 20, 2),
            of="TSim", seed=123, simrep=1, ts=True):  # Throws out Concurrent when done
    """
    Parallelized simulations with lambda such that
    rho = (d*lambda)/(mu*N) and mu = 1
    <=> lambda = (N*rho)/d.
    """
    run = Concurrent(maxtime, rho, d, r, order, seed)
    run.WriteEach(of=[of], simrep=simrep, ts=ts)
    return run.Results()


In [8]:
# def tabled(to_print = [[0,1],[1,2]]):
#     for row in to_print:
#         line=""
#         for _ in row:
#             line+=" {:<15}"
#         print(line.format(*row))






In [9]:
# Requires a lot of refactoring
simrep=ncpus # == # cpus allotted to limit amount held in memory
ToOrder = [5, 25, 50, 100] # <- WARNING: values too high => memory leaks! Future: stream directly to file, del local
def rearrange_for_test(results, which = 0, ToOrder=ToOrder):
    per_queue = {q: [] for q in range(ToOrder[which])}
    for sim in range(simrep):
        sim_results=results[sim][0]["TimeQueueSize"]
        queue_times = {q:{} for q in range(ToOrder[which])}
        for time, value in sim_results.items():
            for queue in range(ToOrder[which]):
                queue_times[queue][time] = value[queue]

        for queue in range(ToOrder[which]):
            per_queue[queue].append(queue_times[queue])
    return per_queue
def arr_t(X):
    return np.array(X).transpose()
gc.collect() # <- from imports, etc

In [10]:
seed = 204
with open(f"largetimeres{time.time()}.txt", "w") as ls:
    for r in [0.8, 0.9, 0.99]:
        ls.write(f"\n================rho={r}, seed={seed}================\n")
        ls.flush()
        for i in range(len(ToOrder)):
            results = SafeRun(of="TSim", order=[ToOrder[i]], maxtime=1000, rho = r, simrep=simrep, ts=False, seed=seed)
            results = results[list(results.keys())[0]] #
            sim_per_queue = rearrange_for_test(results,i)
            del results
            gc.collect()
            test, thresh=hsic.dHSIC_resample_test([arr_t(hsic.time_sampler(sim_per_queue[m], range(900,1000,5), 1000)) for m in range(ToOrder[i])], 500)
            print(thresh)
            print(test)
            ls.write(f"\nFor N={ToOrder[i]}, rho = {r}\n, T=range(900,1000,5)"
                  f"Reject null (H_0: independence): {test <= thresh if thresh!=None else (False if test != 1 else True)}\n"
                  f"----------------------------------------\n"
                  f"HSIC statistic: {round(test,3)}\n"
                  f"Critical value: {thresh}\n"
                  f"alpha: {0.05}\n")
            ls.flush()
            del sim_per_queue
            gc.collect()
    ls.close()


In [11]:
exit()
